# ЭТАП 1. ОБРАБОТКА И АНАЛИЗ ДАННЫХ

1.1. Загрузка данных, получение первичной информации и определение типов признаков.

Извлечем данные о пользователях

In [18]:
import pandas as pd, dill
from sys import getsizeof
from numpy import uint8

data_train = pd.read_csv('data_train.csv')
data_test = pd.read_csv('data_test.csv')

print(data_train.head())
print(data_test.head())

   Unnamed: 0       id  vas_id    buy_time  target
0           0   540968     8.0  1537131600     0.0
1           1  1454121     4.0  1531688400     0.0
2           2  2458816     1.0  1534107600     0.0
3           3  3535012     5.0  1535922000     0.0
4           4  1693214     1.0  1535922000     0.0
   Unnamed: 0       id  vas_id    buy_time
0           0  3130519     2.0  1548018000
1           1  2000860     4.0  1548018000
2           2  1099444     2.0  1546808400
3           3  1343255     5.0  1547413200
4           4  1277040     2.0  1546808400


Первый столбец в датафреймах - просто номер строки. Уберем его:

In [19]:
data_train = data_train[data_train.columns[1:]]
data_test = data_test[data_test.columns[1:]]

print(data_train.head())
print(data_test.head())

        id  vas_id    buy_time  target
0   540968     8.0  1537131600     0.0
1  1454121     4.0  1531688400     0.0
2  2458816     1.0  1534107600     0.0
3  3535012     5.0  1535922000     0.0
4  1693214     1.0  1535922000     0.0
        id  vas_id    buy_time
0  3130519     2.0  1548018000
1  2000860     4.0  1548018000
2  1099444     2.0  1546808400
3  1343255     5.0  1547413200
4  1277040     2.0  1546808400


Теперь извлечем их признаки (первые 100000 строк):

In [20]:
features = pd.read_csv('features.csv',nrows=100000,delimiter='\t')
features = features[features.columns[1:]]

features.head()

,id,buy_time,0,1,2,3,4,5,6,7,...,243,244,245,246,247,248,249,250,251,252
0,2013026,1531688400,18.910029,46.980888,4.969214,-1.386798,3.791754,-14.01179,-16.08618,-65.076097,...,-977.373846,-613.770792,-25.996269,-37.630448,-301.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
1,2014722,1539550800,36.690029,152.400888,448.069214,563.833202,463.841754,568.99821,-16.08618,-53.216097,...,-891.373846,-544.770792,-20.996269,48.369552,80.252276,-13.832889,-0.694428,-1.175933,-0.45614,0.0
2,2015199,1545598800,-67.019971,157.050888,-63.180786,178.103202,-68.598246,156.99821,3.51382,25.183903,...,-977.373846,-613.770792,-12.996269,-37.630448,10829.252276,-25.832889,-0.694428,-12.175933,-0.45614,0.0
3,2021765,1534107600,7.010029,150.200888,-6.930786,216.213202,76.621754,351.84821,-16.08618,-65.076097,...,-973.373846,-613.770792,-23.996269,-37.630448,-205.747724,-24.832889,-0.694428,-11.175933,-0.45614,1.0
4,2027465,1533502800,-90.439971,134.220888,-104.380786,153.643202,-109.798246,132.53821,-16.08618,-65.076097,...,1643.626154,2007.229208,206.003731,-21.630448,6667.252276,92.167111,-0.694428,49.824067,47.54386,0.0


Определим типы признаков

In [21]:
types = {'int': [], 'float': [], 'string': [], 'other': []}

for i in range(253): #все поля, начиная с 3-го, имеют нумерацию от 0 до 252
    col = features[str(i)]
    x = str(col.dtype)
    
    if x.find('float')>=0:
        types['float'].append(str(i))
    elif x.find('int')>=0:
        types['int'].append(str(i))
    elif x.find('str')>=0:
        types['str'].append(str(i))
    else:
        types['other'].append(str(i))
    
for k in types.keys():
    print('{}: {}'.format(k,len(types[k])))

int: 0
float: 253
string: 0
other: 0


В датафрейме все признаки вещественные. Нужен другой способ определения типа признаков

In [22]:
features_nuniq = features[types['float']].apply(lambda x: x.nunique(dropna=False))
features_nuniq.shape

(253,)

Все признаки из features (за исключением первых двух):

In [23]:
f_all = set(features_nuniq.index.tolist())
len(f_all)

253

Определим признаки с единственным значением (константные):

In [24]:
f_const = set(features_nuniq[features_nuniq == 1].index.tolist())
len(f_const)

8

Определим вещественные признаки:

In [25]:
f_numeric = (features[types['float']].fillna(0).astype(int).sum() - features[types['float']].fillna(0).sum()).abs()
f_numeric = set(f_numeric[f_numeric > 0].index.tolist())
len(f_numeric)

247

Другие признаки

In [26]:
f_other = f_all - (f_numeric | f_const)
len(f_other)

1

бинарные признаки

In [27]:
f_binary = set(features[types['float']].loc[:, f_other].columns[(
                      (features[types['float']].loc[:, f_other].max() == 1) & \
                      (features[types['float']].loc[:, f_other].min() == 0) & \
                      (features[types['float']].loc[:, f_other].isnull().sum() == 0))])
print(len(f_binary))

f_other = f_other - f_binary
print(len(f_other))

0
1


Категориальные признаки

Так не работает (KeyError: "None of [Index(['252'], dtype='object')] are in the [index]"), если отбираем всего один признак (как у меня):

f_categorical = set(features[types['float']].loc[f_other][features[types['float']].loc[f_other] <= 10].index.tolist())

А вот так:

In [28]:
f_categorical = set(features[types['float']][f_other].columns)

f_other = f_other - f_categorical
len(f_other)

0

Класс

In [29]:
f_numeric = f_numeric | f_other
f_numeric = f_numeric - f_const #пофиксил ОШИБКУ: numeric и const в assert считаются отдельно, поэтому необходимо вычесть одно из другого
f_other = f_other - f_numeric
len(f_other)

assert(features_nuniq.shape[0] == len(f_const) + len(f_binary) + len(f_numeric) + len(f_categorical))

#готовые списки признаков для отбора
f_ok = list(f_binary | f_categorical | f_numeric)
f_binary, f_categorical, f_numeric = list(f_binary), list(f_categorical), list(f_numeric)

print(f_binary)
print(f_categorical)
print(f_numeric)

[]
['252']
['26', '191', '69', '153', '97', '174', '63', '243', '105', '72', '186', '33', '201', '71', '94', '224', '28', '4', '99', '10', '214', '197', '208', '96', '44', '34', '247', '244', '29', '80', '108', '65', '241', '42', '109', '193', '19', '220', '167', '161', '0', '210', '12', '86', '32', '148', '107', '127', '155', '250', '180', '173', '68', '59', '22', '61', '136', '60', '8', '151', '182', '237', '101', '36', '246', '92', '52', '54', '11', '111', '134', '124', '47', '202', '226', '146', '194', '235', '93', '126', '141', '121', '150', '104', '130', '187', '74', '164', '195', '192', '103', '58', '90', '123', '31', '142', '50', '230', '62', '116', '56', '91', '66', '211', '162', '228', '188', '27', '67', '129', '79', '163', '185', '207', '140', '215', '156', '209', '206', '41', '40', '39', '6', '45', '48', '73', '87', '135', '231', '236', '18', '117', '213', '114', '170', '165', '152', '95', '158', '179', '131', '229', '176', '240', '14', '132', '225', '159', '233', '234', '1

Проверим признак 252 на бинарность

In [30]:
features['252'].value_counts()

0.0     73952
1.0     26046
82.0        1
10.0        1
Name: 252, dtype: int64

Признак 252 бинарный, поэтому изменим списки признаков:

In [31]:
f_binary = ['252']
f_categorical = []

Сохраним список подходящих полей в файл для дальнейшего использования

In [32]:
with open('cls.dat','wb') as f:
    dill.dump((f_binary,f_categorical,f_numeric),f)

Определим, уникальны ли пользователи в данных, а также определим кол-во тарифов:

In [33]:
cou_users_train = data_train['id'].nunique()
cou_users_test = data_test['id'].nunique()
cou_users_features = features['id'].nunique()

cou_tariffes_train = data_train['vas_id'].value_counts()
cou_tariffes_test = data_test['vas_id'].value_counts()

print(cou_users_train)
print(cou_users_test)
print(cou_users_features,end='\n\n')
print(cou_tariffes_train,end='\n\n')
print(cou_tariffes_test)

806613
70152
99930

1.0    310175
2.0    249505
5.0     94085
4.0     85756
6.0     57878
7.0     15432
8.0     13350
9.0      5472
Name: vas_id, dtype: int64

2.0    31361
5.0    13073
6.0    12976
1.0     8413
4.0     3058
7.0     1192
8.0      619
9.0      539
Name: vas_id, dtype: int64


Cостав тарифных программ одинаков (1,2,4,5,6,7,8,9)

Определим примерное кол-во строк в файле features.csv, т.к. невозможно его прочитать целиком

In [34]:
size_1_s = (getsizeof(features)-144)/len(features) #размер 1 строки
features_cou_str = 22526067261/size_1_s

print(size_1_s)
print(features_cou_str)

2040.0
11042189.83382353


Согласно предварительным подсчетам, файл содержит около 11,04 млн. строк.

Исследуем столбцы id, vas_id, buy_time, target на наличие пропусков ВО ВСЕХ таблицах:

In [35]:
print('Датафрейм data_train:')
for c in list(data_train.columns):
    print('Столбец {} - кол-во nan-значений: {}'.format(c,data_train[c].isna().astype(uint8).sum()))

print('Датафрейм data_test:')
for c in list(data_test.columns):
    print('Столбец {} - кол-во nan-значений: {}'.format(c,data_test[c].isna().astype(uint8).sum()))

print('Датафрейм features:')
for c in ['id','buy_time']:
    print('Столбец {} - кол-во nan-значений: {}'.format(c,features[c].isna().astype(uint8).sum()))

Датафрейм data_train:
Столбец id - кол-во nan-значений: 0
Столбец vas_id - кол-во nan-значений: 0
Столбец buy_time - кол-во nan-значений: 0
Столбец target - кол-во nan-значений: 0
Датафрейм data_test:
Столбец id - кол-во nan-значений: 0
Столбец vas_id - кол-во nan-значений: 0
Столбец buy_time - кол-во nan-значений: 0
Датафрейм features:
Столбец id - кол-во nan-значений: 0
Столбец buy_time - кол-во nan-значений: 0


В таких столбцах пропуска нет, все в порядке.

На следующем шаге создадим пайплайн для преобразования значений признаков, а также с его помощью объединим обучающие и тестовые данные с признаками пользователей в конкретные моменты времени